**License**

 rosenbrock_simple

 Sun Apr 02 21:13:00 2023\
 Copyright  2023\
 Sandro Dias Pinto Vitenti  <vitenti@uel.br>
___
 rosenbrock_simple\
 Copyright (C) 2023 Sandro Dias Pinto Vitenti  <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.

In [ ]:
import sys

from numcosmo_py import Ncm

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import numpy as np
import math

import getdist
from getdist import plots
import pandas as pd
import getdist
from getdist import plots


from numcosmo_py.sampling.apes import APES
from numcosmo_py.sampling.catalog import Catalog
from numcosmo_py.plotting.tools import set_rc_params_article

import emcee
import zeus



In [ ]:
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

ssize=300000
nwalkers = 300
verbose=False

In [ ]:
def log_prob(x, ivar):
    return -0.5 * (100.0 * (x[1] - x[0] * x[0])**2 + (1.0 - x[0])**2) * 1.0e-1

ndim, nwalkers = 2, nwalkers
p0 = np.random.randn(nwalkers, ndim)

In [ ]:
sampler_apes = APES(nwalkers=nwalkers, ndim=ndim, model=None, log_prob=log_prob, args=[0])
sampler_apes.run_mcmc(p0, ssize // nwalkers)
mcat_apes = sampler_apes.get_catalog()
mcat_apes.trim(50)

In [ ]:
sampler_emcee = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[0])
state_emcee = sampler_emcee.run_mcmc(p0, ssize // nwalkers)
chain_emcee = sampler_emcee.get_chain(flat=True)
log_prob_emcee = sampler_emcee.get_log_prob(flat=True)
mcat_emcee = Catalog(ndim=ndim, nwalkers=nwalkers, run_type="EMCEE")
mcat_emcee.add_points_m2lnp(chain_emcee, -2.0 * log_prob_emcee)
mcat_emcee.trim(50)

In [ ]:
sampler_zeus = zeus.EnsembleSampler(nwalkers, ndim, log_prob, args=[0], verbose=verbose)
sampler_zeus.run_mcmc(p0, ssize // nwalkers, progress=False)
chain_zeus = sampler_zeus.get_chain(flat=True)
log_prob_zeus = sampler_zeus.get_log_prob(flat=True)
mcat_zeus = Catalog(ndim=ndim, nwalkers=nwalkers, run_type="ZEUS")
mcat_zeus.add_points_m2lnp(chain_zeus, -2.0 * log_prob_zeus, interweaved=False)
mcat_zeus.trim(50)

In [ ]:
sigma = np.sqrt([10.0, 2401/10])
Ncm.cfg_msg_sepa()
mcat_apes.print_status()
Ncm.cfg_msg_sepa()
mcat_emcee.print_status()
Ncm.cfg_msg_sepa()
mcat_zeus.print_status()

In [ ]:
mean = np.array([1.0, 11.0])
print(np.abs(mcat_apes.get_mean()/mean - 1.0))
print(np.abs(mcat_emcee.get_mean()/mean - 1.0))
print(np.abs(mcat_zeus.get_mean()/mean - 1.0))

In [ ]:
print(np.abs(np.sqrt(np.diagonal(mcat_apes.get_covar()))/sigma - 1.0))
print(np.abs(np.sqrt(np.diagonal(mcat_emcee.get_covar()))/sigma - 1.0))
print(np.abs(np.sqrt(np.diagonal(mcat_zeus.get_covar()))/sigma - 1.0))

In [ ]:
mcs_apes = mcat_apes.get_mcsamples("APES")
mcs_emcee = mcat_emcee.get_mcsamples("EMCEE")
mcs_zeus = mcat_zeus.get_mcsamples("ZEUS")

In [ ]:
set_rc_params_article(ncol=2)

g = getdist.plots.get_subplot_plotter(width_inch=plt.rcParams["figure.figsize"][0])
g.settings.linewidth = 0.01
g.triangle_plot([mcs_apes, mcs_emcee, mcs_zeus], shaded=True)
